# **Setup**

In [4]:
!pip install tensorflow
!pip install pyyaml h5py 
!pip install pretty_midi
!pip3 install scamp
!pip3 install fluidsynth
!pip3 install python-rtmidi


In [32]:

import collections
import datetime
import glob
import numpy as np
import pathlib
import pandas as pd
import pretty_midi
import seaborn as sns
import tensorflow as tf

from IPython import display
from matplotlib import pyplot as plt
from typing import Dict, List, Optional, Sequence, Tuple

import os

from tensorflow import keras

from scamp import*

#from google.colab import files
#from supercollider import Server, Synth

In [9]:
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

# Sampling rate for audio playback
_SAMPLING_RATE = 16000

data_dir = pathlib.Path('data/maestro-v2.0.0')
if not data_dir.exists():
  tf.keras.utils.get_file(
      'maestro-v2.0.0-midi.zip',
      origin='https://storage.googleapis.com/magentadata/datasets/maestro/v2.0.0/maestro-v2.0.0-midi.zip',
      extract=True,
      cache_dir='.', cache_subdir='data',
  )

filenames = glob.glob(str(data_dir/'**/*.mid*'))
print('Number of files:', len(filenames))
sample_file = filenames[1]
print(sample_file)
pm = pretty_midi.PrettyMIDI(sample_file)

instrument = pm.instruments[0]
instrument_name = pretty_midi.program_to_instrument_name(instrument.program)

59243107/59243107 [==============================] - 4s 0us/step
Number of files: 1282
data\maestro-v2.0.0\2004\MIDI-Unprocessed_SMF_02_R1_2004_01-05_ORIG_MID--AUDIO_02_R1_2004_06_Track06_wav.midi


In [10]:
def midi_to_notes(midi_file: str) -> pd.DataFrame:
  pm = pretty_midi.PrettyMIDI(midi_file)
  instrument = pm.instruments[0]
  notes = collections.defaultdict(list)

  # Sort the notes by start time
  sorted_notes = sorted(instrument.notes, key=lambda note: note.start)
  prev_start = sorted_notes[0].start

  for note in sorted_notes:
    start = note.start
    end = note.end
    notes['pitch'].append(note.pitch)
    notes['start'].append(start)
    notes['end'].append(end)
    notes['step'].append(start - prev_start)
    notes['duration'].append(end - start)
    prev_start = start

  return pd.DataFrame({name: np.array(value) for name, value in notes.items()})

In [11]:
def display_audio(pm: pretty_midi.PrettyMIDI, seconds=30):
  waveform = pm.fluidsynth(fs=_SAMPLING_RATE)
  # Take a sample of the generated waveform to mitigate kernel resets
  waveform_short = waveform[:seconds*_SAMPLING_RATE]
  return display.Audio(waveform_short, rate=_SAMPLING_RATE)

num_files = 5
all_notes = []
for f in filenames[:num_files]:
  notes = midi_to_notes(f)
  all_notes.append(notes)

all_notes = pd.concat(all_notes)

n_notes = len(all_notes)

In [12]:
raw_notes = midi_to_notes(sample_file)

key_order = ['pitch', 'step', 'duration']
train_notes = np.stack([all_notes[key] for key in key_order], axis=1)
notes_ds = tf.data.Dataset.from_tensor_slices(train_notes)
notes_ds.element_spec

TensorSpec(shape=(3,), dtype=tf.float64, name=None)

In [13]:
def mse_with_positive_pressure(y_true: tf.Tensor, y_pred: tf.Tensor):
  mse = (y_true - y_pred) ** 2
  positive_pressure = 10 * tf.maximum(-y_pred, 0.0)
  return tf.reduce_mean(mse + positive_pressure)

In [119]:
seq_length = 5 #Do not change
vocab_size = 128

input_shape = (seq_length, 3)
learning_rate = 0.005

inputs = tf.keras.Input(input_shape)
x = tf.keras.layers.LSTM(128)(inputs)

outputs = {
  'pitch': tf.keras.layers.Dense(128, name='pitch')(x),
  'step': tf.keras.layers.Dense(1, name='step')(x),
  'duration': tf.keras.layers.Dense(1, name='duration')(x),
}

model = tf.keras.models.load_model('C://Users//erik9//OneDrive//Documents//RNN_models//RNN_Model_1', custom_objects={"mse_with_positive_pressure": mse_with_positive_pressure})



In [120]:
def predict_next_note(
    notes: np.ndarray, 
    keras_model: tf.keras.Model, 
    temperature: float = 1.0) -> int:
  """Generates a note IDs using a trained sequence model."""

  assert temperature > 0

  # Add batch dimension
  inputs = tf.expand_dims(notes, 0)

  predictions = model.predict(inputs)
  pitch_logits = predictions['pitch']
  step = predictions['step']
  duration = predictions['duration']

  pitch_logits /= temperature
  pitch = tf.random.categorical(pitch_logits, num_samples=1)
  pitch = tf.squeeze(pitch, axis=-1)
  duration = tf.squeeze(duration, axis=-1)
  step = tf.squeeze(step, axis=-1)

  # `step` and `duration` values should be non-negative
  step = tf.maximum(0, step)
  duration = tf.maximum(0, duration)

  return int(pitch), float(step), float(duration)

In [121]:
def notes_to_midi(
  notes: pd.DataFrame,
  out_file: str, 
  instrument_name: str,
  velocity: int = 100,  # note loudness
) -> pretty_midi.PrettyMIDI:

  pm = pretty_midi.PrettyMIDI()
  instrument = pretty_midi.Instrument(
      program=pretty_midi.instrument_name_to_program(
          instrument_name))

  prev_start = 0
  for i, note in notes.iterrows():
    start = float(prev_start + note['step'])
    end = float(start + note['duration'])
    note = pretty_midi.Note(
        velocity=velocity,
        pitch=int(note['pitch']),
        start=start,
        end=end,
    )
    instrument.notes.append(note)
    prev_start = start

  pm.instruments.append(instrument)
  pm.write(out_file)
  return pm

# **Note Generation**

In [122]:
s = Session()
to_sc = s.new_osc_part("cloud", port = 57120);
#to_sc = s.new_osc_part("organ", port = 57120);

In [125]:
import time

temperature = 3.0
num_predictions = 10

#Duration and step is set to 0.5 as the current model fails to create non-zero values for these.
dur = 0.5
stp = 0.5

#Create your own input!
input_notes = ([(26, 0, dur), (86, stp, dur), (26, stp, dur), (46, stp, dur), (36, stp, dur)])

generated_notes = []
prev_start = 0

to_sc.play_note(i[0]+70,0,duration)
#Adding the input to generated notes
for i in input_notes:
  
  duration = dur # TODO! CHANGE
  step = stp     # TODO! CHANGE
  start = prev_start + step
  end = start + duration
  generated_notes.append((i[0], step, duration, start, end))
  to_sc.play_note(i[0]+70,0.1,duration)
  prev_start = start

for _ in range(num_predictions):
  t = time.time()

  pitch, step, duration = predict_next_note(input_notes, model, temperature)
  
  duration = dur # TODO! CHANGE
  step = stp     # TODO! CHANGE

  start = prev_start + step
  end = start + duration
  input_note = (pitch, step, duration)

  generated_notes.append((*input_note, start, end))
  input_notes = np.delete(input_notes, 0, axis=0)
  input_notes = np.append(input_notes, np.expand_dims(input_note, 0), axis=0)
  prev_start = start
    
  to_sc.play_note(pitch+70,0.1,duration)

  print(f"pitch: {pitch}")
  print(f"Note Nr. {len(generated_notes)}")
  print(f"Time: {time.time()-t}")

generated_notes = pd.DataFrame(
    generated_notes, columns=(*key_order, 'start', 'end'))


1/1 [==============================] - 0s 16ms/step
pitch: 80
Note Nr. 6
Time: 0.042333364486694336
1/1 [==============================] - 0s 22ms/step
pitch: 87
Note Nr. 7
Time: 0.050284624099731445
1/1 [==============================] - 0s 19ms/step
pitch: 92
Note Nr. 8
Time: 0.04659008979797363
1/1 [==============================] - 0s 19ms/step
pitch: 89
Note Nr. 9
Time: 0.046393394470214844
1/1 [==============================] - 0s 18ms/step
pitch: 89
Note Nr. 10
Time: 0.04871988296508789
1/1 [==============================] - 0s 16ms/step
pitch: 87
Note Nr. 11
Time: 0.04429173469543457
1/1 [==============================] - 0s 18ms/step
pitch: 89
Note Nr. 12
Time: 0.04986095428466797
1/1 [==============================] - 0s 16ms/step
pitch: 54
Note Nr. 13
Time: 0.044109344482421875
1/1 [==============================] - 0s 18ms/step
pitch: 92
Note Nr. 14
Time: 0.04504966735839844
1/1 [==============================] - 0s 20ms/step
pitch: 89
Note Nr. 15
Time: 0.0479886531829834


# **Waste**

In [29]:
#out_file = 'output_nm.mid'
#t = time.time()
#out_pm = notes_to_midi(
#    generated_notes, out_file=out_file, instrument_name=instrument_name)
#print(time.time()-t)
#files.download(out_file)

0.005997419357299805
